In [11]:
# %load src/header.py
'''LOAD'''
# %load /Users/rokushou/Desktop/header.py
'''GENERAL'''
import os
#import time
#import timeit
#import warnings
#warnings.simplefilter('ignore')
import random
import collections
#import secrets
import math
import itertools
'''DATA'''
import numpy as np
import pandas as pd
import databricks.koalas as ks
'''MONGO'''
#from pymongo import MongoClient
'''PLOT'''
import matplotlib.pyplot as plt
plt.style.use('seaborn-pastel')
font = {'size':16}
import seaborn as sns
'''SCIPY/STATS'''
#import scipy.stats as scs
from scipy.optimize import nnls
#import statsmodels.api as sm
'''SCIKIT LEARN'''
#from sklearn.datasets import make_classification, load_iris, load_boston, load_digits
#from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, r2_score, mean_squared_error, classification_report, make_scorer
#from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
#from sklearn.linear_model import LinearRegression, LogisticRegression
#from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier, AdaBoostRegressor, AdaBoostClassifier
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import KMeans
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
'''TENSORFLOW'''
#import tensorflow as tf
'''NLTK'''
#import nltk
#import unicodedata
#import string
#from nltk.corpus import stopwords
#from nltk.tokenize import sent_tokenize, word_tokenize
#from nltk.stem.porter import PorterStemmer
#from nltk.stem.snowball import SnowballStemmer
#from nltk.stem.wordnet import WordNetLemmatizer
'''NETWORK'''
import networkx as nx
import nxpd
import community as comm
'''FUNCTIONS'''
nrange = lambda x : itertools.repeat(None, x)
def argpsort(a, k):
    return np.argpartition(a, range(k))[:k]
def arghsort(a, k):
    b = np.argpartition(a, k)[:k]
    return b[np.argsort(a[b])]
'''PALETTE'''
t = 'setsuna'
qan = {'setsuna':'quanta'}
twi = '#DCB8E7' #Pale, light grayish mulberry
twi_blu = '#273873' #Dark sapphire blue
twi_pur = '#662D8A' #Moderate purple
twi_pnk = '#ED438D' #Brilliant raspberry
'''RETURN'''
!fortune | cowsay -f dragon

 _________________________________________ 
/ If I could stick my pen in my heart, I  \
| would spill it all over the stage.      |
| Would it satisfy ya, would it slide on  |
| by ya, Would you think the boy was      |
| strange? Ain't he strange? ... If I     |
| could stick a knife in my heart,        |
| Suicide right on the stage, Would it be |
| enough for your teenage lust, Would it  |
| help to ease the pain? Ease your brain? |
|                                         |
| -- Rolling Stones, "It's Only Rock'N    |
\ Roll"                                   /
 ----------------------------------------- 
      \                    / \  //\
       \    |\___/|      /   \//  \\
            /0  0  \__  /    //  | \ \    
           /     /  \/_/    //   |  \  \  
           @_^_@'/   \/_   //    |   \   \ 
           //_^_/     \/_ //     |    \    \
        ( //) |        \///      |     \     \
      ( / /) _|_ /   )  //       |      \     _\
    ( // /) '/,_ _ _/  ( ; -.    |    _ 

# Initial Cleaning

I have a directory of csvs from stat.ink which contains player submitted match data. I want to process them into one cleaned database. Leveraging the power of Koalas, I can use a Spark backend with a Pandas-like interface. I will first clean one csv to devolop a pipeline.

In [43]:
# Gather list of raw data filenames
rawlst = !ls data/raw
# Load one to EDA
df = pd.read_csv('data/raw/'+rawlst[0])
df.head(3)

,# period,game-ver,lobby-mode,lobby,mode,stage,time,win,knockout,A1-weapon,...,B3-level,B4-weapon,B4-kill-assist,B4-kill,B4-assist,B4-death,B4-special,B4-inked,B4-rank,B4-level
0,2017-08-07T04:00:00+00:00,1.1.2,gachi,squad_2,hoko,ama,300,alpha,False,splatscope,...,29,hokusai,7,6,1,5,4,1353,b-,33
1,2017-08-08T02:00:00+00:00,1.1.2,gachi,standard,yagura,hokke,221,alpha,True,splatscope,...,22,96gal,6,5,1,8,1,736,b,27
2,2017-08-08T02:00:00+00:00,1.1.2,gachi,standard,yagura,chozame,312,bravo,False,splatscope,...,19,nzap85,10,5,5,9,3,696,b+,24


### Simple Feature Selection

Since this data is from match results and I want to create a predictive model for win rate, there are many features that cannot be used as they are only knowable at the end of a match:

- period: when the match was played
    - irrelevant because this is not a timeseries model
    - version number better covers patch changes
- time: how long the match took
- knockout: if the match was won through the objective as opposed to a time-out
- player kill/assist/death/special/inked: player statistics calculated at the end of each match

In [44]:
# initialize list with first features
drop_lst = ['# period','time', 'knockout']
# concatanate player statistics features for each player
for player in ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4']:
    drop_lst += [player+'-kill-assist', player+'-kill', player+'-assist',
                 player+'-death', player+'-special', player+'-inked']
# drop features in list
df.drop(drop_lst, axis=1, inplace=True)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 30 columns):
game-ver      26 non-null object
lobby-mode    26 non-null object
lobby         26 non-null object
mode          26 non-null object
stage         26 non-null object
win           26 non-null object
A1-weapon     26 non-null object
A1-rank       26 non-null object
A1-level      26 non-null int64
A2-weapon     26 non-null object
A2-rank       26 non-null object
A2-level      26 non-null int64
A3-weapon     23 non-null object
A3-rank       26 non-null object
A3-level      26 non-null int64
A4-weapon     25 non-null object
A4-rank       26 non-null object
A4-level      26 non-null int64
B1-weapon     24 non-null object
B1-rank       26 non-null object
B1-level      26 non-null int64
B2-weapon     25 non-null object
B2-rank       26 non-null object
B2-level      26 non-null int64
B3-weapon     24 non-null object
B3-rank       26 non-null object
B3-level      26 non-null int64
B4-weapon    

In [21]:
df['# period'] = pd.to_datetime(df['# period'])

In [22]:
df.iloc[:, 0:10].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 10 columns):
# period      26 non-null datetime64[ns, UTC]
game-ver      26 non-null object
lobby-mode    26 non-null object
lobby         26 non-null object
mode          26 non-null object
stage         26 non-null object
time          26 non-null int64
win           26 non-null object
knockout      26 non-null bool
A1-weapon     26 non-null object
dtypes: bool(1), datetime64[ns, UTC](1), int64(1), object(7)
memory usage: 1.9+ KB


In [23]:
df.head()

,# period,game-ver,lobby-mode,lobby,mode,stage,time,win,knockout,A1-weapon,...,B3-level,B4-weapon,B4-kill-assist,B4-kill,B4-assist,B4-death,B4-special,B4-inked,B4-rank,B4-level
0,2017-08-07 04:00:00+00:00,1.1.2,gachi,squad_2,hoko,ama,300,alpha,False,splatscope,...,29,hokusai,7,6,1,5,4,1353,b-,33
1,2017-08-08 02:00:00+00:00,1.1.2,gachi,standard,yagura,hokke,221,alpha,True,splatscope,...,22,96gal,6,5,1,8,1,736,b,27
2,2017-08-08 02:00:00+00:00,1.1.2,gachi,standard,yagura,chozame,312,bravo,False,splatscope,...,19,nzap85,10,5,5,9,3,696,b+,24
3,2017-08-08 02:00:00+00:00,1.1.2,gachi,standard,yagura,chozame,300,alpha,False,splatscope,...,20,hokusai,20,12,8,10,2,702,b,28
4,2017-08-09 00:00:00+00:00,1.1.2,gachi,standard,hoko,chozame,304,alpha,False,sputtery,...,33,sputtery,6,3,3,9,5,1252,s+,33


In [24]:
df['# period'][0]

Timestamp('2017-08-07 04:00:00+0000', tz='UTC')